# model for idenifying breaking job
looking to train a model using Baudilio Tejerina jobs, in order to make a model that idenifies outliers.

model jobs have SLURM_JOB_NAME: N2O4

In [56]:
# epmt_query contains the EPMT Query API
import epmt_query as eq
# epmt_outliers contains the EPMT Outlier Detection API
import epmt_outliers as eod
# epmt_stat contains statistical functions
import epmt_stat as es
import pandas as pd
import numpy as np
import time

In [57]:
# potential criteria are before/after job start/end/creation times, 
# potential tags are exp_component, exp_name, exp_time, exp_platform, and exp_target (i.e. command line args to frepp)
# potential output formats are dict, pandas, terse, and orm
username = 'Baudilio.Tejerina'
jobs_all = eq.get_jobs(fltr=(eq.Job.user_id == username),
                       fmt='dict')
print(f'number of elements in jobs_all={len(jobs_all)}')
jobs_all[0]['start']

number of elements in jobs_all=1756


datetime.datetime(2024, 1, 5, 14, 29, 26, 521879)

In [58]:
#sort jobs into desired category
N2O4_jobs = []
not_N2O4_jobs = []
for job_instance in range(len(jobs_all)):
    if jobs_all[job_instance]['env_dict']['SLURM_JOB_NAME'][0:4] == 'N2O4':
        N2O4_jobs.append(jobs_all[job_instance])
    else:
        not_N2O4_jobs.append(jobs_all[job_instance])

In [59]:
feature_list=['cpu_time','duration','rssmax','num_procs'] #parameters used to identify outliers
feature_list_wjobid=['jobid']+feature_list
exp_comp_outlier_jobs = eod.detect_outlier_jobs(N2O4_jobs, features=feature_list, sanity_check = False)

# the output is a 2-element list: first element is a panda's dataframe, second is a dict
# we're only interested in the datafram for now.
df=exp_comp_outlier_jobs[0]
df['score'] = 0
df['score'] = df.sum(axis=1)
sorted_df = df.sort_values('score', ascending=False)

In [60]:
#here we sort based on score to find inliers
# univariate classifier, default features
exp_comp_outlier_jobs = eod.detect_outlier_jobs(N2O4_jobs, sanity_check = False, features = feature_list)
df = exp_comp_outlier_jobs[0]

# create a df column called `score` to hold the sum
df['score'] = 0
df['score'] = df.sum(axis=1)

# select only jobs whose scores are 0.
inliers_df = df[df.score == 0][feature_list_wjobid]

# count how many jobs, inliers we have
num_inliers = len(inliers_df)
model_jobs = list(inliers_df.jobid.values)
print(f'num_inliers={num_inliers} out of {len(df)} possible jobs')

num_inliers=37 out of 61 possible jobs


In [61]:
#time to make the actual model
start = time.time()
r = eq.create_refmodel(model_jobs, methods=es.mvod_classifiers(), sanity_check = False) #create trained model
print('time: ',(time.time()-start)/60, ' minutes')

time:  0.003466490904490153  minutes


In [63]:
df, model_scores = eod.detect_outlier_jobs(not_N2O4_jobs, methods=es.mvod_classifiers(), trained_model=r['id'], sanity_check = False)
df_sorted=df.sort_values('outlier', ascending=False)
df_sorted


ValueError: dataframe columns (['rchar', 'syscr', 'syscw', 'wchar', 'cstime', 'cutime', 'majflt', 'minflt', 'rssmax', 'cmajflt', 'cminflt', 'inblock', 'outblock', 'usertime', 'processor', 'starttime', 'vol_ctxsw', 'guest_time', 'read_bytes', 'systemtime', 'time_oncpu', 'timeslices', 'invol_ctxsw', 'write_bytes', 'time_waiting', 'delayacct_blkio_time', 'cancelled_write_bytes']) contain atlleast one NaN each